## Extracción

In [ ]:

import pandas as pd
import json

# Cargar datos desde el archivo JSON local
with open("TelecomX_Data.json", "r") as file:
    data = json.load(file)

# Convertir a DataFrame
df = pd.json_normalize(data)

# Vista inicial
print(df.shape)
df.head()


## Transformación

In [ ]:

# Renombrar columnas para mayor claridad
df.columns = df.columns.str.replace("customer.", "")
df.columns = df.columns.str.replace("phone.", "")
df.columns = df.columns.str.replace("internet.", "")
df.columns = df.columns.str.replace("account.", "")
df.columns = df.columns.str.replace("Charges.", "Charges_")

# Cambiar tipo de datos
df["Charges_Total"] = pd.to_numeric(df["Charges_Total"], errors="coerce")
df["Charges_Monthly"] = pd.to_numeric(df["Charges_Monthly"], errors="coerce")

# Rellenar valores nulos en Churn
df["Churn"] = df["Churn"].replace("", "No")

# Verificar nulls
print(df.isnull().sum())


## Conoce el conjunto de datos

In [ ]:

# Resumen general
df.info()

# Tipos de datos
print(df.dtypes)

# Estadísticas de variables numéricas
print(df.describe())

# Distribución de algunas variables categóricas
for col in ["gender", "Churn", "Contract", "PaymentMethod"]:
    print(f"\nColumna: {col}")
    print(df[col].value_counts())


## Comprobación de incoherencias

In [ ]:

# Valores nulos
print("Valores nulos por columna:\n", df.isnull().sum())

# Valores únicos en categóricas
for col in df.select_dtypes(include="object").columns:
    print(f"\nColumna: {col}")
    print(df[col].unique())

# Duplicados
duplicados = df.duplicated(subset="customerID").sum()
print(f"\nNúmero de registros duplicados: {duplicados}")

# Valores en Churn
print("\nValores en Churn:", df["Churn"].unique())

# Coherencia numérica
print("\nDescripción de variables numéricas:")
print(df[["tenure", "Charges_Monthly", "Charges_Total"]].describe())

# Inconsistencias Charges_Total < Charges_Monthly
inconsistencias = df[df["Charges_Total"] < df["Charges_Monthly"]]
print(f"\nRegistros con inconsistencia en Charges_Total vs Monthly: {len(inconsistencias)}")


## Carga y análisis (EDA)

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# Distribución de Churn
sns.countplot(data=df, x="Churn")
plt.title("Clientes que abandonaron vs. se quedaron")
plt.show()

# Contrato vs Churn
sns.countplot(data=df, x="Contract", hue="Churn")
plt.title("Churn por tipo de contrato")
plt.show()

# Método de pago vs Churn
sns.countplot(data=df, x="PaymentMethod", hue="Churn")
plt.xticks(rotation=45)
plt.title("Churn por método de pago")
plt.show()

# Heatmap de correlación
sns.heatmap(df[["SeniorCitizen", "tenure", "Charges_Monthly", "Charges_Total"]].corr(), annot=True, cmap="coolwarm")
plt.title("Correlación de variables numéricas")
plt.show()


## Informe final


### Hallazgos principales
- Los clientes con contratos mensuales muestran una tasa de churn más alta.
- El churn es mayor en clientes que pagan con Electronic check.
- Clientes con menor tenure tienden a irse más rápido.
- Los cargos mensuales altos, especialmente en fibra óptica, parecen estar asociados a mayor evasión.

### Recomendaciones
- Incentivar contratos de largo plazo con descuentos.
- Ofrecer planes más económicos para clientes nuevos.
- Revisar políticas de facturación en pagos electrónicos.
